## Import modules

In [1]:
classpath.addPath("tinyir-1.1.jar")

In [2]:
import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

In [3]:
// import scala.io.Source  // for importing txt files
import java.io._  // for saving txt files
// import scala.collection.mutable.HashMap  //HashMap used for counting elements in linear time

import java.io._

In [4]:
// import scala.util.Random
import scala.collection.mutable.{Map => MutMap}
// enables "mutable lists"
import scala.collection.mutable.ListBuffer  

import scala.collection.mutable.{Map => MutMap}
import scala.collection.mutable.ListBuffer

## Define classes and functions

In [5]:
def token_filter(text_body: String) = {
    processing.StopWords.filterOutSW(
        processing.Tokenizer.tokenize(text_body.
                                      replaceAll("\\P{L}+", " "))
    ).
    map(x => PorterStemmer.stem(x)).filter(_.trim.nonEmpty).toList
}

defined function token_filter

In [15]:
// Create YMParse for customized tokenization

class YMParse(is: InputStream) extends processing.TipsterParse(is) { 
    override def tokens: List[String] = token_filter(content)
    def hname: Int = name.hashCode()
    def htokens: List[Int] = token_filter(content).map(t => t.hashCode())
  //override def ID: String = name - need to fix the ID.
}

defined class YMParse

In [16]:
import ch.ethz.dal.tinyir.processing.TipsterParse
import ch.ethz.dal.tinyir.processing.Tokenizer
import ch.ethz.dal.tinyir.processing.XMLDocument

// Create YMStream for customized stream using YMParse
class YMStream (path: String, ext: String = "") extends io.ParsedXMLStream(new io.ZipDirStream(path, "")){
  def stream : Stream[YMParse] = unparsed.stream.map(is => new YMParse(is))
  def length = unparsed.length 
}



import ch.ethz.dal.tinyir.processing.TipsterParse
import ch.ethz.dal.tinyir.processing.Tokenizer
import ch.ethz.dal.tinyir.processing.XMLDocument
defined class YMStream

In [17]:
val collection = new YMStream("resources").stream
val collection2 = new io.TipsterStream("resources").stream

collection: Stream[YMParse] = Stream(
  cmd14$$user$YMParse@19d6dbea,
  cmd14$$user$YMParse@4199c280,
  cmd14$$user$YMParse@6fa1e781,
  cmd14$$user$YMParse@41a8f641,
  cmd14$$user$YMParse@58a9dbf7,
  cmd14$$user$YMParse@1b9fd9f9,
  cmd14$$user$YMParse@47999f00,
  cmd14$$user$YMParse@57e8d3f1,
  cmd14$$user$YMParse@ec73470,
  cmd14$$user$YMParse@1fb5a518,
  cmd14$$user$YMParse@7dd6c3cf,
  cmd14$$user$YMParse@2b4bacab,
  cmd14$$user$YMParse@1cbfa1fe,
  cmd14$$user$YMParse@15cc211d,
  cmd14$$user$YMParse@d0bcd5f,
  cmd14$$user$YMParse@17a12e9d,
  cmd14$$user$YMParse@62e786ef,
  cmd14$$user$YMParse@76ea2dd3,
  cmd14$$user$YMParse@16b179ea,
...
collection2: Stream[XMLDocument] = Stream(
  ch.ethz.dal.tinyir.processing.TipsterParse@6d324779,
  ch.ethz.dal.tinyir.processing.TipsterParse@3ebfb480,
  ch.ethz.dal.tinyir.processing.TipsterParse@5d504670,
  ch.ethz.dal.tinyir.processing.TipsterParse@61ee3bfc,
  ch.ethz.dal.tinyir.processing.TipsterParse@508285a0,
  ch.ethz.dal.tinyir.processing.Ti

In [27]:
collection(84493).htokens.size

res26: Int = 287

In [26]:
collection(84493).tokens.size

res25: Int = 287

In [33]:
import ch.ethz.dal.tinyir.processing._
def fwStream (docs: Stream[YMParse]) :
   Stream[(Int,List[Int])]
   = docs.map(d => (d.hname, d.htokens))

import ch.ethz.dal.tinyir.processing._
defined function fwStream

In [34]:
fwStream(collection)

res33: Stream[(Int, List[Int])] = Stream(
  (
    -2143258984,
    List(
      94851467,
      3512060,
      -1266285233,
      1490282602,
      103185,
      109548807,
      3045820,
      -1234877242,
      -342511989,
      2998801,
      92905987,
      2998801,
      108511772,
      -1655966997,
      351160793,
      98619021,
...

In [35]:
def fwIndex (docs: Stream[YMParse]) :
   Map[Int,List[Int]] = fwStream(docs).toMap

defined function fwIndex

In [ ]:
fwIndex(collection.take(50000))

In [ ]:
def postings (s: Stream[YMParse]): Stream[(Int,Int)] =
  s.flatMap( d => d.htokens.map(token => (token,d.hname)) )

In [ ]:
postings(collection).groupBy(_._1).mapValues(_.map(p => p._2).distinct.sorted)

In [ ]:
// Frequency Index

case class TfTuple(term: String, doc: String, count: Int)
def tfTuples (docs: Stream[Document]) : Stream[TfTuple] =
    docs.flatMap( d => d.tokens.groupBy(identity)
    .map{ case (tk,lst) => TfTuple(tk, d.name, lst.length) }
val fqIndex : Map[String,List[(String,Int)]] =
  tfTuples(stream).groupBy(_.term)
  .mapValues(_.map(tfT => (tfT.doc, tfT.count)).sorted)

In [32]:
"b".getBytes

res31: Array[Byte] = Array(98)

In [ ]:
var test = "a" :: "b" :: "c"

In [ ]:
val pos_i = new indexing.PosIndex(collection.take(100))

In [ ]:
val simple_i = new indexing.SimpleIndex(collection2.take(50000))

In [ ]:
collection2(10).ID

In [ ]:
simple_i.index

In [ ]:
val mb = 1024*1024
val runtime = Runtime.getRuntime
println(s"Used Memory:  " + (runtime.totalMemory - runtime.freeMemory) / mb)
println(s"Free Memory:  " + runtime.freeMemory / mb)
println(s"Total Memory: " + runtime.totalMemory / mb)
println(s"Max Memory:   " + runtime.maxMemory / mb)